In [1]:
import numpy as np
import xarray as xr
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
ds = xr.open_dataset('hgt.2023.nc')

In [ ]:
ds_500 = ds.sel(level=500)
Z = ds_500['hgt'].values  # shape: [time, lat, lon]

In [ ]:
num_days, nlat, nlon = Z.shape
input_size = nlat * nlon

# Flatten spatial dimensions
Z_flat = Z.reshape(num_days, input_size)

In [ ]:
# We want to predict Z(t+1) from Z(t)
X = Z_flat[:-1]       # All days except the last
y = Z_flat[1:]        # From the second day to the end

# Split into training and test (for example, first 70% train, next 20% val, last 10% test)
train_split = int(0.7 * len(X))
val_split = int(0.9 * len(X))

X_train, y_train = X[:train_split], y[:train_split]
X_val, y_val = X[train_split:val_split], y[train_split:val_split]
X_test, y_test = X[val_split:], y[val_split:]

In [ ]:

# Normalize data (important for NN training)
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)
y_train_norm = scaler.transform(y_train)
y_val_norm = scaler.transform(y_val)
y_test_norm = scaler.transform(y_test)

In [ ]:

model = Sequential()
# Simple architecture with two hidden layers
model.add(Dense(256, activation='relu', input_shape=(input_size,)))
model.add(Dense(128, activation='relu'))
# Output layer: same size as input (predict the entire global field)
model.add(Dense(input_size))

model.compile(optimizer='adam', loss='mse')


In [ ]:

# -------------------------------------------------------
# Train the Model
# -------------------------------------------------------
history = model.fit(X_train_norm, y_train_norm,
                    
                    
                    validation_data=(X_val_norm, y_val_norm),
                    epochs=10,  # Increase epochs as needed
                    batch_size=32)